# Week 2 Assignment Classifiers

Prem Sagar Malothu

22B0052

#WELCOME
In this Assignment, you'll use **Classifier** to predict the output. You are allowed to write/edit between the
```# START CODE HERE``` & ```# END CODE HERE``` only.<br>
Don't edit the test cells otherwise you'd be failed in the assignment.

Upload the ```utils.zip``` file before running the 1st cell.
[Demo](https://drive.google.com/file/d/1ebkQR6ikjGottmxa5KL83ZWdjTLvRxbm/view?usp=share_link)

**BEST OF LUCK**

**RUN THIS CODE TO UNIZP THE REQUIRED FILES**

In [2]:
!unzip -j "/content/drive/MyDrive/Learner Space/ML/Assignment_2/utils.zip" -d.

Archive:  /content/drive/MyDrive/Learner Space/ML/Assignment_2/utils.zip
  inflating: ./weather_classification_data.csv  
  inflating: ./data_scaled_labeled_check.csv  


# Dataset Loading and Pre-Processing

In [3]:
import pandas as pd
data = pd.read_csv("/content/weather_classification_data.csv")

## Data Info
```data.info``` gives info about data-types and number of null objects.  
There are no null data, no need to remove any.

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13200 entries, 0 to 13199
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Temperature           13200 non-null  float64
 1   Humidity              13200 non-null  int64  
 2   Wind Speed            13200 non-null  float64
 3   Precipitation (%)     13200 non-null  float64
 4   Cloud Cover           13200 non-null  object 
 5   Atmospheric Pressure  13200 non-null  float64
 6   UV Index              13200 non-null  int64  
 7   Season                13200 non-null  object 
 8   Visibility (km)       13200 non-null  float64
 9   Location              13200 non-null  object 
 10  Weather Type          13200 non-null  object 
dtypes: float64(5), int64(2), object(4)
memory usage: 1.1+ MB


## Data Shuffling
Data Shuffling removes any bias present and creates uniform distribution.

In [6]:
from sklearn.utils import shuffle
data = shuffle(data)
data.head()

,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location,Weather Type
7201,-10.0,88,13.5,61.0,overcast,987.50,1,Winter,4.0,inland,Snowy
1467,22.0,74,5.5,25.0,partly cloudy,1019.50,2,Spring,7.0,coastal,Cloudy
11640,26.0,70,9.5,98.0,partly cloudy,995.82,1,Winter,2.0,inland,Rainy
9496,12.0,79,9.0,48.0,partly cloudy,1015.25,1,Winter,8.5,inland,Cloudy
13123,1.0,79,2.5,83.0,overcast,999.67,1,Winter,2.5,inland,Snowy


## Scaling and Labeling
Machine can't understand string so all the object datatypes need to be LABELED with integers.<br><br>
Why SCALING?<br>
You might have noticed one column contains data with large values (of order 1000) whereas some column contain small value data (of order 10). As a result, large value columns get more power/importance than others. To remove this disparity we scaled them to same order/values.

**CODE**

We need to find the columns with object datatypes.

HINT: Use ```data.select_dtypes().columns.tolist()```<br>Reference: [w3schools](https://www.w3schools.com/python/pandas/ref_df_select_dtypes.asp#:~:text=The%20select_dtypes()%20method%20returns,the%20specified%20dtype(s).&text=Note%3A%20You%20must%20specify%20at,you%20will%20get%20an%20error.)

In [8]:
# START CODE HERE
object_columns = data.select_dtypes(include="object").columns.tolist()
non_object_columns = data.select_dtypes(exclude="object").columns.tolist()
# END CODE HERE
print(f"Object Columns: {object_columns}\nNon Object Columns: {non_object_columns}")

Object Columns: ['Cloud Cover', 'Season', 'Location', 'Weather Type']
Non Object Columns: ['Temperature', 'Humidity', 'Wind Speed', 'Precipitation (%)', 'Atmospheric Pressure', 'UV Index', 'Visibility (km)']


**TEST** (DON'T EDIT THIS CELL)

In [9]:
# DON'T EDIT THIS CELL
if (object_columns == ['Cloud Cover', 'Season', 'Location', 'Weather Type'] and non_object_columns == ['Temperature', 'Humidity', 'Wind Speed', 'Precipitation (%)', 'Atmospheric Pressure', 'UV Index', 'Visibility (km)']):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


**CODE**

We need to convert the object-type columns data into integral labels and scale the non-object-type columns data.

HINT: Use ```ColumnTransformer()```<br>Reference: [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), [ChatGPT](https://chatgpt.com/c/7f3c1b02-5da0-4c34-bdb2-d999dccb9c2f)

In [11]:
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

# START CODE HERE

column_transformer = ColumnTransformer([
    ("cat_transform", OrdinalEncoder(), object_columns),
    ("scaler", StandardScaler(), non_object_columns)
])

# Fit and transform the data
data_scaled_labeled = column_transformer.fit_transform(data)

# END CODE HERE

all_columns = object_columns + non_object_columns

# Create the new DataFrame
data_scaled_labeled = pd.DataFrame(data_scaled_labeled, columns=all_columns)

**TEST** (DON'T EDIT THIS CELL)

In [12]:
# DON'T EDIT THIS CELL

data_scaled_labeled_check = pd.read_csv("/content/data_scaled_labeled_check.csv")
import numpy as np
data_scaled_labeled_values = np.sort(data_scaled_labeled.values,axis=0)
data_scaled_labeled_check_values = np.sort(data_scaled_labeled_check.values,axis=0)
if np.allclose(data_scaled_labeled_values, data_scaled_labeled_check_values, equal_nan=True):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


## Data Splitting

**CODE**

We need to drop the Weather Type column from data_scaled_labeled to get the X and pick Weather Type column to get Y

Use 90-10 split of data, i.e. 90% for training, 10% for testing

In [14]:
data_scaled_labeled.head()

,Cloud Cover,Season,Location,Weather Type,Temperature,Humidity,Wind Speed,Precipitation (%),Atmospheric Pressure,UV Index,Visibility (km)
0,2.0,3.0,1.0,2.0,-1.675379,0.955217,0.530916,0.230256,-0.492709,-0.779410,-0.433923
1,3.0,1.0,0.0,0.0,0.165218,0.261924,-0.627087,-0.896669,0.367548,-0.520104,0.455922
2,3.0,3.0,1.0,1.0,0.395293,0.063841,-0.048086,1.388485,-0.269043,-0.779410,-1.027154
3,3.0,3.0,1.0,0.0,-0.409968,0.509529,-0.120461,-0.176689,0.253295,-0.779410,0.900845
4,2.0,3.0,1.0,2.0,-1.042674,0.509529,-1.061339,0.918933,-0.165543,-0.779410,-0.878846


In [17]:
X = data_scaled_labeled.drop("Weather Type", axis = 1)
y = data_scaled_labeled["Weather Type"]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.9, random_state = 10)

**TEST** (DON'T EDIT THIS CELL)

In [18]:
# DON'T EDIT THIS CELL

if (len(X_train)==11880 and len(X_test)==1320):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

if (len(y_train)==11880 and len(y_test)==1320):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed
Test Passed


In [19]:
X_train.head()

,Cloud Cover,Season,Location,Temperature,Humidity,Wind Speed,Precipitation (%),Atmospheric Pressure,UV Index,Visibility (km)
7017,3.0,3.0,2.0,0.567848,-0.480889,-0.192836,-1.428828,0.169689,1.295035,0.455922
12278,0.0,1.0,0.0,0.970479,-1.768434,-0.120461,-1.679256,0.528577,1.813646,0.010999
2859,2.0,2.0,2.0,0.855442,-0.877057,0.024290,-0.959276,-0.145918,-0.779410,0.010999
5495,3.0,3.0,1.0,0.107699,-0.431369,-0.120461,-0.176689,0.282866,-0.779410,0.604230
8469,2.0,1.0,2.0,-0.179894,0.757134,0.603291,0.261560,0.101674,-0.260799,-1.027154


# Training and Testing
Use any suitable classifier to fit the training data and check the accuracy on test data

Don't forget to import that library

**CODE**

ACCURACY should be more than 90%,

1. Fit the model to X_train and y_train
2. Find the model score on X_test and y_test

# 1. SVM Model

In [23]:
# START CODE HERE

from sklearn import svm

model =svm.SVC()
model.fit(X_train,y_train)
score = model.score(X_test,y_test)

# END CODE HERE

print("Accuracy:", score*100)

Accuracy: 91.28787878787878


# 2. RandomForest Model

In [30]:
# START CODE HERE

from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X_train,y_train)
score = model.score(X_test,y_test)

# END CODE HERE

print("Accuracy:", score*100)

Accuracy: 91.66666666666666


# 3. K-Nearest Neighbour Model

In [34]:
# START CODE HERE

from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors = 4)
model.fit(X_train,y_train)
score = model.score(X_test,y_test)

# END CODE HERE

print("Accuracy:", score*100)

Accuracy: 91.28787878787878


**TEST** (DON'T EDIT THIS CELL)

In [24]:
# DON'T EDIT THIS CELL

if (score>0.90):
  print(f"\033[32mTest Passed\033[0m")
else:
  print(f"\033[31mTest Failed\033[0m")

Test Passed


# Predicting Output (Here, Weather Type) (Optional)

In [25]:
input = X_test.head()
input

,Cloud Cover,Season,Location,Temperature,Humidity,Wind Speed,Precipitation (%),Atmospheric Pressure,UV Index,Visibility (km)
11923,3.0,3.0,1.0,-0.985155,0.658092,1.254668,0.605898,-0.345659,-1.038715,-0.582231
2331,3.0,3.0,2.0,-0.294931,-0.233285,-0.988963,-0.239296,0.022101,-0.001493,0.900845
217,2.0,3.0,1.0,0.740404,1.103780,0.313790,0.825022,0.059468,-0.520104,-1.027154
1760,2.0,3.0,1.0,-1.330267,1.153301,1.254668,0.511987,-0.636265,-1.038715,-0.433923
3963,0.0,1.0,0.0,0.855442,-0.579931,-0.409962,-1.241007,0.451154,1.035729,0.010999


In [26]:
y = model.predict(input)
y

array([2., 0., 1., 2., 3.])

Here, the outputs are in numbers. We can perform inverse column transfer for getiing the string values.
